# Design an optimized supply chain network

**Author:** Devanshi Verma
<br>
**Date**: August-15-2020
<br>
**Goal**: Determine regional production at a plant such that we have the balance between high capacity and low capacity plants while mainting regional demands and low costs

**Modelling**

- Production at regional facilities: Low and High
- Production to other regions
- Productional facilities open or close

**Decision Variables**

 $$X_{i,j} = \text{Quantity produced at i and shipped to j}$$
<br>
$$\begin{Bmatrix}
Y_{i,s}=1 & \text{ if plant location i and capacity s is open} 
\\
 Y_{i,s}=0 & \text{ if plant location i and capacity s is closed}
\end{Bmatrix}\ $$



**Objective Function**
$$min \sum _{i=1}^{n} f_{is}y_{is} +\sum _{i=1}^{n}\sum _{i=1}^{m} v_{ij}x_{ij}$$
<br>
- i.e. Sum of (fixed costs * Yis) over production facilities + Sum of (variablecost X xij) over productional facilities and number of markets


**Type of Optimization problem**: Mixed Integer programming since X is continous whereas Y is constrained to integer values.

In [268]:
#importing the libraries
import pandas as pd
from pulp import *

In [269]:
#importing the datasets
demand=pd.read_csv("Data/Demand.csv")
fix_cost=pd.read_csv("Data/fix_cost.csv")
var_cost=pd.read_csv("Data/var_cost.csv")
cap=pd.read_csv("Data/cap.csv")

In [270]:
#Setting indexes
fix_cost=fix_cost.set_index("Supply_Region")
var_cost=var_cost.set_index("Supply_Region")
demand=demand.set_index("Supply_Region")
cap=cap.set_index("Supply_Region")

**Datasets**

In [271]:
fix_cost.head()

,Low_Cap,High_Cap
Supply_Region,,
U.S,6500,9500
Brazil,3230,4730
Canada,4980,7270
Mexico,1000,1460
Argentina,1200,1752


In [272]:
demand.head()

,Dmd
Supply_Region,
U.S,769013
Brazil,157293
Canada,79000
Mexico,48767
Argentina,35446


In [273]:
var_cost.head()

,U.S,Brazil,Canada,Mexico,Argentina,Chile,Colombia,Puerto Rico,Uruguay
Supply_Region,,,,,,,,,
U.S,0.6,7.0,2.3,1.9,4.0,3.2,4.1,3.5,4.60
Brazil,7.0,0.6,9.2,7.0,2.8,1.5,3.2,4.0,1.00
Canada,2.3,9.2,0.6,3.6,11.2,5.3,6.4,5.4,5.50
Mexico,1.9,7.0,3.6,0.7,7.4,3.7,3.7,3.7,4.00
Argentina,4.0,2.8,11.2,7.4,0.5,0.6,7.0,6.3,0.65


In [274]:
cap.head()

,Low_Cap,High_Cap
Supply_Region,,
U.S,500.0,1500
Brazil,500.0,1500
Canada,500.0,1500
Mexico,500.0,1500
Argentina,500.0,1500


**Model**

In [275]:
#Initialise the model
model=LpProblem("Capacitedplantlocation",LpMinimize)

#Decision Variables
loc=list(demand.index)
size=['Low_Cap','High_Cap']

x=LpVariable.dicts("production_", [(i,j) for i in loc for j in loc],lowBound=0,upBound=None,cat='Continuous')
y=LpVariable.dicts("plant_", [(i,s) for i in loc for s in size],cat='Binary')

#Objective Function
model += (lpSum([fix_cost.loc[i,s] * y[(i,s)] for s in size for i in loc])
        + lpSum([var_cost.loc[i,j] * x[(i,j)] for i in loc for j in loc]))

# Define the constraints
for j in loc:
    model += lpSum([x[i,j] for i in loc]) == demand.loc[j,'Dmd']
    
for i in loc:
    model += lpSum([x[(i, j)] for j in loc]) <= lpSum([cap.loc[i,s] * y[i,s] for s in size])
    
model.solve()

print("Status of the mode is: {}".format(LpStatus[model.status]))

Status of the mode is: Infeasible


**Output**

In [276]:
import re
for i in model.variables():
    print(i,i.varValue)

plant__('Argentina',_'High_Cap') 1.0
plant__('Argentina',_'Low_Cap') 1.0
plant__('Brazil',_'High_Cap') 1.0
plant__('Brazil',_'Low_Cap') 1.0
plant__('Canada',_'High_Cap') 1.0
plant__('Canada',_'Low_Cap') 1.0
plant__('Chile',_'High_Cap') 1.0
plant__('Chile',_'Low_Cap') 1.0
plant__('Colombia',_'High_Cap') 1.0
plant__('Colombia',_'Low_Cap') 1.0
plant__('Mexico',_'High_Cap') 1.0
plant__('Mexico',_'Low_Cap') 1.0
plant__('Puerto_Rico',_'High_Cap') 1.0
plant__('Puerto_Rico',_'Low_Cap') 1.0
plant__('U.S',_'High_Cap') 1.0
plant__('U.S',_'Low_Cap') 2239.04
plant__('Uruguay',_'High_Cap') 1.0
plant__('Uruguay',_'Low_Cap') 1.0
production__('Argentina',_'Argentina') 0.0
production__('Argentina',_'Brazil') 2000.0
production__('Argentina',_'Canada') 0.0
production__('Argentina',_'Chile') 0.0
production__('Argentina',_'Colombia') 0.0
production__('Argentina',_'Mexico') 0.0
production__('Argentina',_'Puerto_Rico') 0.0
production__('Argentina',_'U.S') 0.0
production__('Argentina',_'Uruguay') 0.0
productio

In [277]:
#converting the output into a pandas dataframe to be used for Travelling Salesman Problem
loc1=[]
loc2=[]
quan=[]
for i in loc:
    for j in loc:
        loc1.append(i)
        loc2.append(j)
        quan.append(x[i,j].varValue)

d=pd.DataFrame(loc1,columns=['location1'])
d['location2']=loc2
d['quantity']=quan
d   

,location1,location2,quantity
0,U.S,U.S,769013.0
1,U.S,Brazil,148516.0
2,U.S,Canada,77000.0
3,U.S,Mexico,46767.0
4,U.S,Argentina,35446.0
...,...,...,...
76,Uruguay,Argentina,0.0
77,Uruguay,Chile,0.0
78,Uruguay,Colombia,0.0
79,Uruguay,Puerto Rico,0.0


In [280]:
loc1=[]
cap_p=[]
status=[]
for i in loc:
    for s in size:
        loc1.append(i)
        cap_p.append(s)
        status.append(y[i,s].varValue)
a=pd.DataFrame(loc1,columns=['location'])
a['capacity']=cap_p
a['status']=status
a 

,location,capacity,status
0,U.S,Low_Cap,2239.04
1,U.S,High_Cap,1.00
2,Brazil,Low_Cap,1.00
3,Brazil,High_Cap,1.00
4,Canada,Low_Cap,1.00
5,Canada,High_Cap,1.00
6,Mexico,Low_Cap,1.00
7,Mexico,High_Cap,1.00
8,Argentina,Low_Cap,1.00
9,Argentina,High_Cap,1.00
